In [23]:
import sqlite3

conn = sqlite3.connect('facts.db')
cur = conn.cursor()

In [24]:
cur.execute("""
DROP TABLE IF EXISTS facts
""")
cur.execute("""
DROP TABLE IF EXISTS tags
""")
cur.execute("""
DROP TABLE IF EXISTS facts_to_tags
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS facts 
(fact_id int PRIMARY KEY, question text, var1 text, var2 text, var3 text, var4 text, right_answer text, fact_text text, source text)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS tags 
(tag_id int PRIMARY KEY, tag text)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS facts_to_tags 
(id INTEGER PRIMARY KEY AUTOINCREMENT, fact_id int, tag_id int)
""")

conn.commit()

In [25]:
import pandas as pd

df = pd.read_csv('facts.csv')[df.columns[0:10]]

In [26]:
df.head()

,Вопрос,Вариант1,Вариант2,Вариант3,Вариант4,Правильный (номер варианта),Объяснение (факт),Теги,Автор(ы),Источник
0,В какой строчке все слова написаны верно?,"Жымолость, шина","Жимолость, шина","Шына, жымолость","Жимолость, шына",2,ЖИ-ШИ пишется с буквой И! Но почему?\nНаписани...,орфография школьные_правила,Настя Чижикова,NaN
1,В какой строчке все ударения выделены верно?,"ЗвонИт, вклЮчим, заключИм","ПозвОнишь, включАтся, звонИм","ВключИт, перезвонИт, переключИшь","ВклЮчатся, звонИт, переклЮчишь",3,Нормативное ударение во всех этих глаголах - н...,орфоэпия школьные_правила,Настя Чижикова,NaN
2,"Выберите все значения, которые может иметь сло...",кричать,плакать,звать на помощь,смеяться,14,"Глаголы орать, верещать, вопить являются сленг...",сленг,Елизавета Клыкова,NaN
3,"Выберите все значения, которые может иметь сло...",смех,зов на помощь,крик,плач,13,"Слово ""ор"" - сленговое выражение, обозначающее...",сленг,Елизавета Клыкова,NaN
4,Вычеркните лишнее слово:,лол,лмао,кринж,ор,3,Лишнее слово - кринж: остальные три слова испо...,сленг заимствования,Елизавета Клыкова,NaN


In [27]:
db_tags = []

for fact_id, row in df.iterrows():
    tags = row['Теги'].split()
    for tag in tags:
        if tag not in db_tags:
            db_tags.append(tag)
            cur.execute('INSERT INTO tags VALUES (?, ?)', (len(db_tags), tag))
            cur.execute('INSERT INTO facts_to_tags (fact_id, tag_id) VALUES (?, ?)', (fact_id+1, len(db_tags)))
            conn.commit()
        else:
            cur.execute('INSERT INTO facts_to_tags (fact_id, tag_id) VALUES (?, ?)', (fact_id+1, db_tags.index(tag)+1))
            
    cur.execute("INSERT INTO facts VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                   (fact_id+1, row['Вопрос'], row['Вариант1'], row['Вариант2'], row['Вариант3'], row['Вариант4'], 
                    row['Правильный (номер варианта)'], row['Объяснение (факт)'], row['Источник']))

In [28]:
conn.commit()